In [1]:
import requests
import numpy as np


def download_data(url,filename):
    result = requests.get(url)
    result.raise_for_status()
    with open(filename,'wb') as FILE:
        for chunk in result.iter_content(102400):
            FILE.write(chunk)
def readout(filename):
    with open(filename,'r') as FILE:
        n = 0
        Dx = []
        Dy = []
        for chunk in FILE:
            X= chunk.split()   #split the line into a list of string
            X= [ float(X[j]) for j in range(len(X)) ]  #convert the string to numbers   
            X.insert(0,1)
            Dx.append(X[0:len(X)-1])     #gather all data ## index len(X)-1 will not be included here
            Dy.append(X[len(X)-1])       #index len(X)-1 only
            n=n+1
        x = np.array(Dx)  #change list X into array x
        y = np.array(Dy)
    print(x,n)            
    return x,y,n
            
def errornum(datasize,xtest,ytest,w):
    er=0

    for i in range(datasize):        
        if w.dot(xtest[i]) != 0:
            if np.sign(w.dot(xtest[i]))!=ytest[i]:   
                er=er+1
        else:
            if ytest[i] != -1:
                er=er+1
    return er


            
url01 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_15_train.dat'
url03 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_18_train.dat'
url02 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_math/hw1_18_test.dat'
download_data(url01, 'data_set.txt')
download_data(url02, 'training_data.txt')
download_data(url03, 'test_data.txt')

In [2]:
trainx, trainy, trainn = readout('training_data.txt')
testx, testy, testn = readout('test_data.txt')

[[1.       0.62926  0.32783  0.010417 0.73102 ]
 [1.       0.32368  0.61439  0.42097  0.025626]
 [1.       0.15968  0.83346  0.97515  0.32762 ]
 ...
 [1.       0.34445  0.30929  0.049222 0.49242 ]
 [1.       0.040789 0.28334  0.55817  0.60612 ]
 [1.       0.36302  0.34961  0.12651  0.54315 ]] 500
[[1.       0.94544  0.42842  0.79833  0.16244 ]
 [1.       0.85365  0.084168 0.5682   0.49221 ]
 [1.       0.17095  0.82127  0.98444  0.51486 ]
 ...
 [1.       0.078827 0.3823   0.13965  0.82677 ]
 [1.       0.92288  0.86313  0.22939  0.92214 ]
 [1.       0.50315  0.41884  0.026094 0.91623 ]] 500


In [3]:
#Pocket algorithm with purely random chosen x

i = 0
j = 0
N = 2000
fail = 0
success = 0
rounds = 0
update = 0
updates = 0
eta = 1
err = 0
maxupdate = 50
n = trainn

for times in range(N):    
    random = np.random.choice(range(n), size=n) #隨機採樣
    xrandom = np.array([trainx[random[j],:] for j in range(n)])
    yrandom = np.array([trainy[random[j]] for j in range(n)])
    xtest = np.array([testx[random[j],:] for j in range(n)])
    ytest = np.array([testy[random[j]] for j in range(n)])
    success=0
    update = 0
    w = np.array([0.0,0.0,0.0,0.0,0.0])
    w1, w2 = w, w+1
    s0=0
    s=0
    rounds = 0
    
    while update<maxupdate:
        success=0
        fail=0
        s0=s
        w2=w1
        
        
        for i in range(n):
            if w.dot(xrandom[i]) != 0:
                if np.sign(w.dot(xrandom[i]))!=yrandom[i]:   
                    w= w+eta*yrandom[i]*xrandom[i]
                    fail=fail+1
                else:
                    success=success+1
            else:
                if yrandom[i] != -1:
                    w= w+eta*yrandom[i]*xrandom[i]
                    fail=fail+1
                else:
                    success=success+1
            i=i+1    
            rounds=rounds+1
            if update+fail>=maxupdate:
                break            
        s=success
        
        if s>=s0 or rounds==0:   #'pocket' the better w       
            w1=w
        
        update=update+fail
#        print(f,f0,w1[1],w2[1],update,updates)


    
    err = err+errornum(n,xtest,ytest,w1)    #check error rate of w1 with test data in every cycle
    
#    updates = updates + update
#    print(times+1, 'th cycle done')
#ave = updates/N
print('average error rate is', err/N/n, ',and maxupdate value is',update)

average error rate is 0.356288 ,and maxupdate value is 50


In [4]:
i = 0
j = 0
N = 2000
fail = 0
success = 0
rounds = 0
update = 0
updates = 0
eta = 1
err = 0
maxupdate = 100
n = trainn

for times in range(N):    
    random = np.random.choice(range(n), size=n) #隨機採樣
    xrandom = np.array([trainx[random[j],:] for j in range(n)])
    yrandom = np.array([trainy[random[j]] for j in range(n)])
    xtest = np.array([testx[random[j],:] for j in range(n)])
    ytest = np.array([testy[random[j]] for j in range(n)])
    success=0
    update = 0
    w = np.array([0.0,0.0,0.0,0.0,0.0])

    while update<maxupdate:
        success=0
        fail=0
        for i in range(n):
            if w.dot(xrandom[i]) != 0:
                if np.sign(w.dot(xrandom[i]))!=yrandom[i]:   
                    w= w+eta*yrandom[i]*xrandom[i]
                    fail=fail+1
                else:
                    success=success+1
            else:
                if yrandom[i] != -1:
                    w= w+eta*yrandom[i]*xrandom[i]
                    fail=fail+1
                else:
                    success=success+1
            i=i+1    
            rounds=rounds+1
            if update+fail>=maxupdate:
                break
        update=update+fail
#        print(w,i,fail,success,update,rounds)
    updates = updates + update

    err = err+errornum(n,xtest,ytest,w1)    #check error rate of w1 with test data in every cycle
    

print('average error rate is', err/N/n,', and maxupdate value is',update)
#ave = updates/N


average error rate is 0.604483 , and maxupdate value is 100


In [5]:
type(range(5)[0])

int

In [6]:
w = np.array([0.0,0.0,0.0,0.0,0.0])
w1=w+1